<a href="https://colab.research.google.com/github/daniel-hain/paper_skill_development_NO/blob/main/10_embeddings_sbert_skills.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies needed for this notebook

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!pip install -qU transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, util

In [16]:
import pickle
import pandas as pd
import numpy as np

import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq

In [5]:
embeddings_dir = "/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/embeddings_folder"

---

## Using sentence-transformers

In [6]:
model = SentenceTransformer('NbAiLab/nb-sbert-base')

# Embeddings Skills

In [7]:
# Read the CSV file
df_skills = pd.read_csv("/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/text_skills_plain.csv")

In [8]:
df_skills.head()

,skill_id,text
0,0005c151-5b5a-4a66-8aac-60e734beb1ab,Tildele og administrere personaloppgaver på om...
1,00064735-8fad-454b-90c7-ed858cc993f2,Føre tilsyn med driften av fengsler eller andr...
2,000709ed-2be5-4193-b056-45a97698d828,"Identifisere undertrykkelse i samfunn, økonomi..."
3,0007bdc2-dd15-4824-b7d6-416522c46f35,"Inspisere rullende materiell, komponenter og s..."
4,00090cc1-1f27-439e-a4e0-19a87a501bfc,Identifisere de forskjellige tjenestene som er...


In [9]:
embeddings = model.encode(df_skills['text'].tolist(), show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/435 [00:00<?, ?it/s]

In [21]:
df_embeddings = pd.DataFrame(embeddings.cpu())

In [22]:
df_embeddings = df_embeddings.set_index(df_skills['skill_id'])

In [23]:
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
skill_id,,,,,,,,,,,,,,,,,,,,,
0005c151-5b5a-4a66-8aac-60e734beb1ab,0.723989,-0.901305,1.309278,-0.466964,0.205073,0.084210,0.255169,-0.139916,-0.041395,-0.322087,...,0.256888,-0.243751,-0.117670,0.161558,-1.131392,0.573538,0.441574,0.461744,-0.089957,0.392363
00064735-8fad-454b-90c7-ed858cc993f2,-0.010961,-0.154471,0.412948,-0.144281,-0.659671,0.628459,-0.148856,0.541366,-0.695440,0.369303,...,-0.254814,-0.157896,0.190320,-0.474529,0.425783,-0.635713,-0.059051,-0.228554,0.343224,0.352786
000709ed-2be5-4193-b056-45a97698d828,-0.163043,-0.016034,0.545689,0.112307,-0.460055,0.596168,0.254476,0.150688,-0.696650,-0.509793,...,0.292366,-0.186682,-0.184541,0.080621,-0.256179,-0.381200,1.101955,-0.698128,0.849906,-0.267852
0007bdc2-dd15-4824-b7d6-416522c46f35,-0.000597,-0.745139,0.613709,-0.140171,0.374975,0.689990,0.616258,-0.357698,-0.612571,-0.056079,...,0.169837,-0.327125,-0.436577,0.321618,-0.480304,0.936462,0.087399,0.248086,-0.054677,0.212425
00090cc1-1f27-439e-a4e0-19a87a501bfc,0.674840,-0.620949,-0.226397,-0.523009,-0.609080,0.866970,0.141116,-0.552859,-1.154060,-0.718580,...,0.120221,0.107177,-0.285379,0.069144,-0.402782,0.321842,0.435917,-0.001722,0.803619,0.888647


In [24]:
df_embeddings = pa.Table.from_pandas(df_embeddings)
pq.write_table(df_embeddings, f"{embeddings_dir}/skill_embeddings.parquet")

In [25]:
# Save as pickle (alternative)
with open(f"{embeddings_dir}/00_skills_embeddings.pickle", 'wb') as pkl:
    pickle.dump(embeddings, pkl)

#with open(f"{embeddings_dir}/skills_embeddings.pickle", 'rb') as pkl:
 #   doc_embedding = pickle.load(pkl)